In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime, date
from pathlib import Path


In [2]:
import local_db

connection = local_db.connection()


In [3]:
today = dt.date.today()
today_str = today.strftime("%Y%m%d")
print(today_str)


20190514


In [4]:
if today<(dt.date(today.year, 9, 1)):
    acad_yr = today.year
else:
    acad_yr = today.year - 1

print(f"acad_yr={acad_yr:d}")

acad_yr=2019


In [5]:
begin_year = "2009"


In [6]:
# read ACADEMIC data
sql_str = (
    "SELECT PEOPLE_CODE_ID, ACADEMIC_YEAR, ACADEMIC_TERM, "
    + "ACADEMIC_SESSION, POPULATION, INQUIRY_FLAG, "
    + "APPLICATION_FLAG, APPLICATION_DATE, "
    + "APP_STATUS, APP_STATUS_DATE, "
    + "APP_DECISION, APP_DECISION_DATE "
    + "FROM ACADEMIC WHERE "
    + f"ACADEMIC_YEAR >= '{begin_year}' "
)
academic = pd.read_sql_query(sql_str, connection)


In [7]:
adm_df = academic.loc[
    ~(academic["POPULATION"].isin(["ADVSTU", "NOND"]))
    & ((academic["INQUIRY_FLAG"] == "Y") | (academic["APPLICATION_FLAG"] == "Y"))
]

adm_df = adm_df.loc[
    (
        (adm_df["ACADEMIC_TERM"].isin(["FALL", "SPRING"]))
        & (adm_df["ACADEMIC_SESSION"] == "MAIN")
        & (adm_df["ACADEMIC_YEAR"] >= "2009")
    )
]

print(adm_df.shape)

(29780, 12)


In [8]:
adm_df = adm_df.loc[
        (adm_df["APP_STATUS"] == "500")
]

print(adm_df.shape)

(3803, 12)


In [9]:
adm_df.head()

,PEOPLE_CODE_ID,ACADEMIC_YEAR,ACADEMIC_TERM,ACADEMIC_SESSION,POPULATION,INQUIRY_FLAG,APPLICATION_FLAG,APPLICATION_DATE,APP_STATUS,APP_STATUS_DATE,APP_DECISION,APP_DECISION_DATE
29,P000000564,2009,FALL,MAIN,,N,Y,2009-02-18 00:00:00,500,2009-02-18,ACC,2009-02-18
160,P000001518,2010,FALL,MAIN,,N,Y,2010-08-10 00:00:00,500,2010-08-10,ACC,2010-08-10
769,P000006539,2009,FALL,MAIN,,N,Y,2009-05-11 00:00:00,500,2009-05-11,ACC,2009-05-11
835,P000007069,2009,SPRING,MAIN,,N,Y,2009-01-22 00:00:00,500,2009-01-22,ACC,2009-01-22
875,P000007236,2011,FALL,MAIN,ACE,N,Y,2011-07-11 00:00:00,500,2011-07-11,ACC,2011-07-11


In [10]:
adm_df = adm_df.rename(
    columns={"APP_STATUS": "field_value", "APP_STATUS_DATE": "create_date"}
)
adm_df.loc[:, "field_name"] = "Application Status"
adm_df = adm_df.loc[~adm_df["create_date"].isnull()]


In [11]:
# create new fields
adm_df["year_term"] = (
    adm_df["ACADEMIC_YEAR"] + "." + adm_df["ACADEMIC_TERM"].str.title()
)
week_number = (
    lambda r: (r["create_date"].date().isocalendar()[1])
    if (r["create_date"].date() >= date((int(r["ACADEMIC_YEAR"]) - 1), 9, 1))
    else (date((int(r["ACADEMIC_YEAR"]) - 1), 9, 1).isocalendar()[1])
)
adm_df["Week_Number"] = adm_df.apply(week_number, axis=1)

# convert ACADEMIC_YEAR to numeric keep numeric-valued records
adm_df["ACADEMIC_YEAR"] = pd.to_numeric(
    adm_df["ACADEMIC_YEAR"], errors="coerce", downcast="integer"
)
adm_df = adm_df.loc[adm_df["ACADEMIC_YEAR"].notnull()]

adm_week_number = (
    lambda r: (
        r["Week_Number"] 
        - (date((int(r["ACADEMIC_YEAR"]) - 1), 9, 1).isocalendar()[1]) 
        + 1
    )
    if (
        r["Week_Number"] >= (date((int(r["ACADEMIC_YEAR"]) - 1), 9, 1).isocalendar()[1])
    )
    else (
        53
        + r["Week_Number"]
        - (date((int(r["ACADEMIC_YEAR"]) - 1), 9, 1).isocalendar()[1])
    )
)
adm_df["Admissions_Week"] = adm_df.apply(adm_week_number, axis=1)

print(adm_df.shape)

(3803, 16)


In [12]:
print(adm_df.columns)

Index(['PEOPLE_CODE_ID', 'ACADEMIC_YEAR', 'ACADEMIC_TERM', 'ACADEMIC_SESSION',
       'POPULATION', 'INQUIRY_FLAG', 'APPLICATION_FLAG', 'APPLICATION_DATE',
       'field_value', 'create_date', 'APP_DECISION', 'APP_DECISION_DATE',
       'field_name', 'year_term', 'Week_Number', 'Admissions_Week'],
      dtype='object')


In [13]:
adm_keep_cols = ["PEOPLE_CODE_ID", "year_term", "Admissions_Week", "field_value"]
adm_df = adm_df.loc[(adm_df["year_term"] > "2011.Spring"), adm_keep_cols]
print(adm_df.shape)
adm_df.head()


(2490, 4)


,PEOPLE_CODE_ID,year_term,Admissions_Week,field_value
1716,P000013625,2012.Spring,20,500
1899,P000015215,2012.Spring,21,500
14035,P000023522,2012.Spring,14,500
21461,P000024471,2013.Spring,15,500
21696,P000024505,2012.Fall,46,500


In [14]:
# admissions status table
admission_status = {
    "300": "Applied",
    "ACC": "Accepted",
    "ACXL": "Canceled",
    "CANC": "Canceled",
    "DEF": "Canceled",
    "DEFR": "Canceled",
    "DENY": "Canceled",
    "DPAC": "Deposited",
    "TRDP": "Deposited",
    "TRPD": "Deposited",
    "TRNS": "Accepted",
    "WAIT": "Accepted",
    "500": "Deposited",
    "PEND": "Applied",
    "COMP": "Applied",
}
adm_stat = pd.DataFrame(
    list(admission_status.items()), columns=["field_value", "admission_status"]
)

adm_df1 = (
    pd.merge(adm_df, adm_stat, on=["field_value"], how="left")
    .drop(["field_value"], axis=1)
    .drop_duplicates(
        ["PEOPLE_CODE_ID", "year_term", "Admissions_Week", "admission_status"]
    )
)
adm_df1 = (adm_df1.loc[:,["year_term", "PEOPLE_CODE_ID", "Admissions_Week", "admission_status"]]
    .sort_values(
        ["year_term", "PEOPLE_CODE_ID", "admission_status", "Admissions_Week"]
    ).drop_duplicates(["year_term", "PEOPLE_CODE_ID", "admission_status"], keep="first")
)
print(adm_df1.head())


    year_term PEOPLE_CODE_ID  Admissions_Week admission_status
4   2012.Fall     P000024505               46        Deposited
5   2012.Fall     P000024771               19        Deposited
9   2012.Fall     P000024996               44        Deposited
12  2012.Fall     P000025120               44        Deposited
13  2012.Fall     P000025164               49        Deposited


In [15]:
e = adm_df1.pivot_table(
    index=["year_term", "PEOPLE_CODE_ID"],
#    columns=["admission_status"],
    values=["Admissions_Week"],
)
e = e.fillna(53)

print(e.head())

                          Admissions_Week
year_term PEOPLE_CODE_ID                 
2012.Fall P000024505                   46
          P000024771                   19
          P000024996                   44
          P000025120                   44
          P000025164                   49


In [16]:
e.reset_index().info()
e.T.reset_index().T.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2490 entries, 0 to 2489
Data columns (total 3 columns):
year_term          2490 non-null object
PEOPLE_CODE_ID     2490 non-null object
Admissions_Week    2490 non-null int64
dtypes: int64(1), object(2)
memory usage: 58.4+ KB
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2491 entries, (index, ) to (2019.Spring, P000056585)
Data columns (total 1 columns):
0    2491 non-null object
dtypes: object(1)
memory usage: 46.5+ KB


In [17]:
weeks = range(1, 54)
for w in weeks:
    f = f"{w:02d}"
    e[f] = np.where(e["Admissions_Week"] <= w , 1, 0)

e = e.reset_index().drop(columns=["Admissions_Week"])
e.head()


,year_term,PEOPLE_CODE_ID,01,02,03,04,05,06,07,08,...,44,45,46,47,48,49,50,51,52,53
0,2012.Fall,P000024505,0,0,0,0,0,0,0,0,...,0,0,1,1,1,1,1,1,1,1
1,2012.Fall,P000024771,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
2,2012.Fall,P000024996,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
3,2012.Fall,P000025120,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,2012.Fall,P000025164,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1


In [18]:
summ = e.groupby(["year_term"]).sum()
summ_t = summ.transpose()


In [19]:
summ.head()

,01,02,03,04,05,06,07,08,09,10,...,44,45,46,47,48,49,50,51,52,53
year_term,,,,,,,,,,,,,,,,,,,,,
2012.Fall,3,3,3,3,3,3,3,3,3,3,...,342,347,351,355,362,363,368,369,375,375
2012.Spring,0,0,0,0,0,0,0,0,0,1,...,39,39,39,39,39,39,39,39,39,39
2013.Fall,3,4,4,4,4,4,4,5,6,7,...,274,277,285,290,295,299,302,302,302,302
2013.Spring,0,0,0,0,0,0,1,2,3,3,...,33,33,33,33,33,33,33,33,33,33
2014.Fall,2,2,2,2,2,2,2,2,2,5,...,270,272,277,283,292,299,302,306,311,311


In [20]:
def adm_week(d):
    """
    returns calendar week number and Admissions Week Number for a given date, d
    """
    year = d.year
    if d >= date(year, 9, 1):
        adm_year_start = year
    else:
        adm_year_start = year - 1

    week_number = d.isocalendar()[1]
    adm_start_week = date(adm_year_start, 9, 1).isocalendar()[1]

    if week_number >= adm_start_week:
        adm_week_number = (week_number - adm_start_week) + 1
    else:
        adm_week_number = 53 + (week_number - adm_start_week)

    return (week_number, adm_week_number)


week_number, adm_week_number = adm_week(today)
print(week_number, adm_week_number)


20 38


In [21]:
def adm_week_end_date(awn, acad_yr):
    """
    returns calendar date for the Friday of the given Admissions Week Number, awn
        and academic year, acad_yr
    """
    
    adm_start_date = dt.date((acad_yr-1), 9, 1)
    adm_start_week = adm_start_date.isocalendar()[1]
    
    if awn <= (53 - adm_start_week):
        wn = adm_start_week + awn - 1
        year = acad_yr - 1
    else:
        wn = awn - (53 - adm_start_week) - 1
        year = acad_yr

    week_end = dt.datetime.strptime(f'{year}-{wn}-1', "%Y-%W-%w").date() + dt.timedelta(days=6.9)
    
    return(week_end)

print(today, adm_week_end_date(adm_week_number, acad_yr))

2019-05-14 2019-05-19


In [22]:
summ_t = summ_t.reset_index().rename(columns={"index": "week_num"})
summ_t["wn"] = summ_t.loc[:, ["week_num"]].astype(int)
#summ_t.head()

In [23]:
week_end = (
    lambda r: adm_week_end_date(int(r["wn"]), acad_yr)
)
summ_t["Week_End"] = summ_t.apply(week_end, axis=1)


In [24]:
summ_t = summ_t.drop(columns=["wn", "week_num"], axis=1).set_index("Week_End")

In [25]:
summ_t.head()

year_term,2012.Fall,2012.Spring,2013.Fall,2013.Spring,2014.Fall,2014.Spring,2015.Fall,2015.Spring,2016.Fall,2016.Spring,2017.Fall,2017.Spring,2018.Fall,2018.Spring,2019.Fall,2019.Spring
Week_End,,,,,,,,,,,,,,,,
2018-09-02,3,0,3,0,2,2,1,4,0,2,8,1,2,2,0,1
2018-09-09,3,0,4,0,2,2,1,4,0,3,8,1,2,2,0,1
2018-09-16,3,0,4,0,2,2,1,5,0,3,9,1,2,2,0,1
2018-09-23,3,0,4,0,2,3,1,5,0,3,9,1,2,2,0,1
2018-09-30,3,0,4,0,2,3,1,5,0,3,9,1,2,2,0,1


In [26]:
b_yr = 2012
e_yr = 2019
term = 'Fall'

terms = [f"{y}.{term}" for y in range(b_yr, e_yr+1)]

df_sumt = summ_t.loc[:, terms]

df_sumt.head()


year_term,2012.Fall,2013.Fall,2014.Fall,2015.Fall,2016.Fall,2017.Fall,2018.Fall,2019.Fall
Week_End,,,,,,,,
2018-09-02,3,3,2,1,0,8,2,0
2018-09-09,3,4,2,1,0,8,2,0
2018-09-16,3,4,2,1,0,9,2,0
2018-09-23,3,4,2,1,0,9,2,0
2018-09-30,3,4,2,1,0,9,2,0


In [27]:
print(today_str)
writer = pd.ExcelWriter(
    f"{term}_HistoricNetDeposits_{b_yr:d}-{e_yr:d}_{today_str}.xlsx"
)
df_sumt.to_excel(writer, "Net Deposits")

writer.save()


20190514
